# Psycopg – wykonywanie SQL w Pythonie

Biblioteka służąca do wykonywania kodu SQL na bazie danych z poziomu Pythona. Jest przystosowana wyłącznie do pracy z systemem PostgreSQL. Inne systemy mają swoje odpowiedniki, np.:

- [MySQL](https://pypi.org/project/mysql-connector-python/)
- [Oracle](https://pypi.org/project/oracledb/)


Najnowsza wersja ma numer 3 ([PyPI](https://pypi.org/project/psycopg/)) chociaż cały czas popularna jest wersja 2 ([PyPI](https://pypi.org/project/psycopg2/)).

In [3]:
!which python

/home/patryk/miniconda3/bin/python


In [4]:
!where python

/bin/bash: line 1: where: command not found


In [1]:
import psycopg

In [6]:
conn = psycopg.connect(dbname="task_manager", user="postgres", password="postgres", host="localhost")
cursor = conn.cursor()

## Zapytania SQL

### CREATE

In [7]:
create_query = """CREATE TABLE tasks(
    id SERIAL,
    description TEXT,
    priority SMALLINT,
    is_completed BOOLEAN
)
"""

In [8]:
cursor.execute(create_query)
conn.commit()

In [9]:
create_query = """CREATE TABLE users(
    id SERIAL,
    username VARCHAR(16),
    password VARCHAR(36),
    is_admin BOOLEAN
)
"""

In [10]:
cursor.execute(create_query)
conn.commit()

### INSERT

In [12]:
insert_query = "INSERT INTO tasks(description, priority, is_completed) VALUES('Opis zadania', 2, true)"

In [13]:
cursor.execute(insert_query)
conn.commit()

#### Klauzula `RETURNING`

In [14]:
insert_query = """INSERT INTO tasks (description, priority, is_completed)
                  VALUES ('Zrobić obiad', 3, false) RETURNING *"""

In [15]:
cursor.execute(insert_query)

record = cursor.fetchone()
record

(2, 'Zrobić obiad', 3, False)

In [16]:
conn.commit()

### SELECT

In [17]:
select_query = "SELECT * FROM tasks"

In [18]:
cursor.execute(select_query)
results = cursor.fetchall()

In [19]:
results

[(1, 'Opis zadania', 2, True), (2, 'Zrobić obiad', 3, False)]

**Wiersze jako słowniki**

In [20]:
cursor = conn.cursor(row_factory=psycopg.rows.dict_row)

In [21]:
cursor.execute(select_query)
results = cursor.fetchall()
results

[{'id': 1, 'description': 'Opis zadania', 'priority': 2, 'is_completed': True},
 {'id': 2,
  'description': 'Zrobić obiad',
  'priority': 3,
  'is_completed': False}]

In [26]:
results[0]["description"]

'Opis zadania'

In [27]:
cursor = conn.cursor()

### UPDATE

In [28]:
update_query = """UPDATE tasks SET description='Nauczyć się SQLa'
                  WHERE id=1 RETURNING *"""

In [29]:
cursor.execute(update_query)
record = cursor.fetchone()
record

(1, 'Nauczyć się SQLa', 2, True)

In [30]:
conn.commit()

### DELETE

In [31]:
delete_query = "DELETE FROM tasks WHERE id=2;"
cursor.execute(delete_query)
conn.commit()

### DROP

In [32]:
drop_query = "DROP TABLE users;"

cursor.execute(drop_query)
conn.commit()

## Template'y zapytań i zapobieganie SQL injection 

In [33]:
id_number = '1'

query = f"UPDATE tasks SET priority = 3 WHERE id = {id_number}"
query

'UPDATE tasks SET priority = 3 WHERE id = 1'

In [34]:
cursor.execute(query)
conn.commit()

---

In [35]:
# przygotowanie tabeli do usunięcia

create_query = "CREATE TABLE test();"
cursor.execute(create_query)
conn.commit()

SQL injection:

In [36]:
id_number = '1;  DROP TABLE test;'


query = f"UPDATE tasks SET priority = 4 WHERE id = {id_number}"
query

'UPDATE tasks SET priority = 4 WHERE id = 1;  DROP TABLE test;'

In [37]:
cursor.execute(query)
conn.commit()

---

In [38]:
id_number = '1'

query = "UPDATE tasks SET priority = 5 WHERE id = %s"

cursor.execute(query, (id_number,))
conn.commit()

Gdybyśmy chcieli zobaczyć podgląd takiego zapytania, możemy zrobić to korzystając z następującego feature'a pythonowych stringów:

In [39]:
query % (id_number,)

'UPDATE tasks SET priority = 5 WHERE id = 1'

---

In [40]:
# create_query = "CREATE TABLE test();"
# cursor.execute(create_query)
# conn.commit()

In [41]:
id_number = '1; DROP TABLE test;'

query = "UPDATE tasks SET priority = 6 WHERE id = %s"

cursor.execute(query, (id_number,))

InvalidTextRepresentation: invalid input syntax for type integer: "1; DROP TABLE test;"
CONTEXT:  unnamed portal parameter $1 = '...'

In [42]:
conn.rollback()

> **ZADANIA**

In [ ]:
cursor.close()
conn.close()